In [1]:
import pandas as pd
import numpy as np 

from pyspark.sql import SparkSession
from pyspark import SparkContext

from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer

from sklearn.feature_extraction.text import CountVectorizer
import os
os.getcwd()
# choose a year you want to analyze
year = str(2019)

In [2]:
import pandas as pd
import io
import requests
url="https://raw.githubusercontent.com/stefangabos/world_countries/master/data/en/countries.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [3]:
companies = c
companies['Company'] = companies['name']
companies = companies[['name','alpha2','Company']]

'/h/224/venia/reddit_methods'

In [4]:
companies = pd.read_csv("fortune1000.csv")
companies = companies[companies['Ambiguous'].isna()]
companies = companies.fillna('')

In [4]:
companies=companies.head(100)

In [6]:
vals = companies['Company'].head(100).values
string = ""
for i, val in enumerate(vals):
    if i != len(vals)-1:
        string = string + 'body LIKE "% {}%" OR '.format(val)
    else: 
        string = string + 'body LIKE "% {}%"'.format(val)

In [8]:
string

'body LIKE "% Walmart%" OR body LIKE "% Exxon Mobil%" OR body LIKE "% Berkshire Hathaway%" OR body LIKE "% McKesson%" OR body LIKE "% UnitedHealth Group%" OR body LIKE "% CVS Health%" OR body LIKE "% General Motors%" OR body LIKE "% Ford Motor%" OR body LIKE "% AT&T%" OR body LIKE "% General Electric%" OR body LIKE "% AmerisourceBergen%" OR body LIKE "% Verizon%" OR body LIKE "% Chevron%" OR body LIKE "% Costco%" OR body LIKE "% Fannie Mae%" OR body LIKE "% Kroger%" OR body LIKE "% Walgreens Boots Alliance%" OR body LIKE "% Cardinal Health%" OR body LIKE "% Express Scripts Holding%" OR body LIKE "% J.P. Morgan Chase%" OR body LIKE "% Boeing%" OR body LIKE "% Microsoft%" OR body LIKE "% Bank of America Corp.%" OR body LIKE "% Wells Fargo%" OR body LIKE "% Home Depot%" OR body LIKE "% Citigroup%" OR body LIKE "% Phillips 66%" OR body LIKE "% IBM%" OR body LIKE "% Valero Energy%" OR body LIKE "% Procter & Gamble%" OR body LIKE "% State Farm Insurance Cos.%" OR body LIKE "% Alphabet%" OR b

In [ ]:
companies_spark = spark.createDataFrame(companies)
companies_spark.createOrReplaceTempView("companies")

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "300g").config("spark.driver.maxResultSize", '300g').config("spark.executor.memory", "300g").config('spark.sql.autoBroadcastJoinThreshold', '-1').getOrCreate()

# spark = SparkSession.builder.config("spark.driver.memory", "150g").config("spark.driver.maxResultSize", '0').config("spark.executor.memory", "150g").config('spark.sql.autoBroadcastJoinThreshold', '-1').getOrCreate()
# df = spark.read.load("/comments_2019.parquet")
df = spark.read.load("/ada/data/reddit/parquet/comments_{}.parquet".format(year))
df.createOrReplaceTempView("comments")

comments_sample = spark.sql("""
    select * from comments WHERE body like '%https://www.%' LIMIT 10000000
""")


In [ ]:
# from pyspark.sql.functions import udf, col 
# from pyspark.sql.types import BooleanType

# # Of you can replace `in` with a regular expression
# contains = udf(lambda s, q: q in s, BooleanType())

# like_with_python_udf = (comments_sample.join(companies_spark)
#     .where(contains(col("body"), col("Company")))
#     .select(col("Company"), col("id"), col('subreddit'), col('created_utc'), col('link_id')))
# like_with_python_udf.show()

In [19]:
# yeet = comments_sample.toPandas()

In [4]:
comments_sample.write.parquet("data/urls_comments_full.parquet")

In [5]:
comments_sample = spark.read.load("data/urls_comments_full.parquet")
data = comments_sample.toPandas()

In [7]:
data['url'] = data['body'].str.extract(r"\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b")

In [8]:
data['domain'] = data['url'].str.extract("www.([^\s]+?\.com)")

In [10]:
data2 = data.dropna(subset=['domain'])

In [11]:
url_counts = data2.groupby('domain')['url'].count().sort_values()
url_counts=url_counts[url_counts>20].index


In [12]:
data2=data2[data2['domain'].isin(url_counts)]

In [17]:
data2[['id','parent_id','score','link_id','author','subreddit','created_utc','domain','url']].to_csv("data/url_sample.csv")

In [ ]:
yeet = like_with_python_udf.toPandas()

In [36]:
yeet.to_csv("countries_sample.csv")

In [21]:
x = 'Because it looked like a tricked out Grand Prix.\n\nhttps://en.wikipedia.org/wiki/Pontiac_Grand_Prix#Sixth_generation_(1997%E2%80%932003)'

In [25]:
x.find("wikipedia.org/wiki/")

61

In [11]:
data 
def clean_text(x):
#     x = x.replace('(', '')
#     x = x.replace(')', '')
#     x = x.replace('-','')
#     x = x.replace('\n', ' ')
#     x = x.replace('#','')
    x = x.replace('\\','')
    return x
data['body_clean'] = data['body'].apply(lambda x: clean_text(x))
data['url'] = data['body_clean'].str.extract(r"(imgur.com/[/\!\@i\%\^\*\$\:\wa-zA-Z0-9_\(\-\.]*)(?:&quot)?")
# data['url'] = data['body_clean'].str.extract(r"([/\!\@i\%\^\*\$\:\wa-zA-Z0-9_\(\-\.]*wikipedia.org/wiki/[/\!\@i\%\^\*\$\:\wa-zA-Z0-9_\(\-\.]*)(?:&quot)?")

In [13]:
data['url'] = data['url'].apply(lambda x: str(x)[0:-1] if str(x)[-1] == '.' else x)
data['url'] = data['url'].apply(lambda x: str(x)[1:] if str(x)[0] == '(' else x)

In [19]:
data2 = data.dropna(subset=['url'])

In [212]:
grouped = data.groupby('url')['url'].count().sort_values(ascending=False)
grouped_ = grouped[grouped > 19]

In [213]:
links = grouped_.index
data2 = data[data['url'].isin(links)]

In [214]:
data2

,id,parent_id,score,author_flair_css_class,author_flair_text,link_id,author,subreddit,body,edited,gilded,controversiality,created_utc,distinguished,body_clean,url
0,t1_ehiksag,t1_ehhp158,1,None,None,t3_avtzct,hilti2,todayilearned,You are forgetting one important thing. Trust....,1.551402e+09,0,0,1551398403,None,You are forgetting one important thing. Trust....,https://en.wikipedia.org/wiki/EPR_(nuclear_rea...
3,t1_ehikty8,t1_ehike7z,28,None,None,t3_avw1qj,moldykobold,BlackPeopleTwitter,8.5%\n\nhttps://en.m.wikipedia.org/wiki/Privat...,NaN,0,0,1551398436,None,8.5%\n\nhttps://en.m.wikipedia.org/wiki/Privat...,https://en.m.wikipedia.org/wiki/Private_prison
4,t1_ehikvd9,t1_ehicqzv,2,None,None,t3_avs5x7,cutelyaware,politics,[Race is not a formal taxonomic rank.](https:/...,NaN,0,0,1551398465,None,[Race is not a formal taxonomic rank.](https:/...,https://en.wikipedia.org/wiki/Race_(biology
6,t1_ehikvxo,t3_avuqmi,6,None,None,t3_avuqmi,jim85541,atheism,"Oh you need to read about the ""Red Scare"" and ...",NaN,0,0,1551398476,None,"Oh you need to read about the ""Red Scare"" and ...",https://en.wikipedia.org/wiki/McCarthyism
9,t1_ehikwk8,t1_ehi9vcb,9,None,None,t3_avss2e,veronalady,GCdebatesQT,> The author goes on about how gender doesn't ...,NaN,0,0,1551398489,None,> The author goes on about how gender doesn't ...,https://en.wikipedia.org/wiki/Gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412183,t1_ehikoqq,t1_ehijci7,87,None,None,t3_avwy8q,nizo505,WTF,"Yep, even blanks can kill:\n\nhttps://en.wikip...",NaN,0,0,1551398331,None,"Yep, even blanks can kill:\n\nhttps://en.wikip...",https://en.wikipedia.org/wiki/Jon-Erik_Hexum
2412187,t1_ehikpj0,t1_ehikbn8,1,economist summers image text blue,Buttery Mayos,t3_avo3k5,Barbarossa3141,neoliberal,[You mean the period where the Palestinians we...,NaN,0,0,1551398347,None,[You mean the period where the Palestinians we...,https://en.wikipedia.org/wiki/King_David_Hotel...
2412189,t1_ehikpjh,t1_ehihwln,3,None,None,t3_avxgi9,potato1,politics,"Super common. Nothing nefarious about it, but ...",NaN,0,0,1551398347,None,"Super common. Nothing nefarious about it, but ...",https://en.wikipedia.org/wiki/Code-switching
2412191,t1_ehikq7m,t1_ehiixrb,0,None,None,t3_avsjh9,Molire,worldnews,Atmospheric escape — \nDominant atmospheric e...,NaN,0,0,1551398360,None,Atmospheric escape — \nDominant atmospheric e...,https://en.wikipedia.org/wiki/Atmospheric_escape


In [216]:
data2[['id','parent_id','link_id','author','subreddit','url']].to_csv("data/wikipedia_posts.csv")

In [21]:
data2.groupby('url')['subreddit'].count().sort_values(ascending=False).head(20)

url
imgur.com/a/m3FI3         115302
imgur.com/a/myIAb          79527
imgur.com/a3i5XQ2.jpg      74154
imgur.com/abcdefg.png      61722
imgur.com/.                51722
imgur.com/                 24934
imgur.com/FIpk52f.gifv     21344
imgur.com/kLUlm9W.png      20548
imgur.com/RjrqakK.gifv     18775
imgur.com/DQOlnc6.png      17430
imgur.com/8ZpA7MZ.jpg      15479
imgur.com/PZtQb.gif        15024
imgur.com/9TTfq91.png      14602
imgur.com/i3qOPay.jpg      13085
imgur.com/a/1WL7J          12856
imgur.com/VkFOras.png      12382
imgur.com/a/XUPxWiF        12350
imgur.com/q9Xjv4p          11770
imgur.com/7Mq5iiX.gifv      9207
imgur.com/upload            9102
Name: subreddit, dtype: int64